In [3]:
# Dependencies and Setup
import pandas as pd

# File to Load (Create the path)
school_csv = "schools_complete.csv"
student_csv = "students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data_df = pd.read_csv(school_csv)
student_data_df = pd.read_csv(student_csv)

# Print out each of the csv files for viewing to analyze
student_data_df.head()

Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  
0             66          79  
1             94          61  
2             90          60  
3             67          58  
4             97          84

In [4]:
# Print out each of the csv files for viewing to analyze
school_data_df.head()

School ID            school_name      type  size   budget
0          0      Huang High School  District  2917  1910635
1          1   Figueroa High School  District  2949  1884411
2          2    Shelton High School   Charter  1761  1056600
3          3  Hernandez High School  District  4635  3022020
4          4    Griffin High School   Charter  1468   917500

In [5]:
# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name"])
school_data_complete_df

Student ID       student_name gender grade         school_name  \
0               0       Paul Bradley      M   9th   Huang High School   
1               1       Victor Smith      M  12th   Huang High School   
2               2    Kevin Rodriguez      M  12th   Huang High School   
3               3  Dr. Richard Scott      M  12th   Huang High School   
4               4         Bonnie Ray      F   9th   Huang High School   
...           ...                ...    ...   ...                 ...   
39165       39165       Donna Howard      F  12th  Thomas High School   
39166       39166          Dawn Bell      F  10th  Thomas High School   
39167       39167     Rebecca Tanner      F   9th  Thomas High School   
39168       39168       Desiree Kidd      F  10th  Thomas High School   
39169       39169    Carolyn Jackson      F  11th  Thomas High School   

       reading_score  math_score  School ID      type  size   budget  
0                 66          79          0  District  2917  1910635  
1                 94          61          0  District  2917  1910635  
2                 90          60          0  District  2917  1910635  
3                 67          58          0  District  2917  1910635  
4                 97          84          0  District  2917  1910635  
...              ...         ...        ...       ...   ...      ...  
39165             99          90         14   Charter  1635  1043130  
39166             95          70         14   Charter  1635  1043130  
39167             73          84         14   Charter  1635  1043130  
39168             99          90         14   Charter  1635  1043130  
39169             95          75         14   Charter  1635  1043130  

[39170 rows x 11 columns]

In [6]:
total_students = school_data_complete_df["student_name"].count()
total_schools = school_data_df["school_name"].count()
total_budget = school_data_df["budget"].sum()
avg_math = student_data_df["math_score"].mean()
avg_read = student_data_df["reading_score"].mean()

# Use loc to find how many students passed math and reading and both
math_pass = (school_data_complete_df.loc[school_data_complete_df["math_score"] >= 70, :])
count_math = math_pass["Student ID"].count()

read_pass = (school_data_complete_df.loc[school_data_complete_df["reading_score"] >= 70, :])
count_read = read_pass["Student ID"].count()

both_pass = (school_data_complete_df.loc[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70), :])
count_both = both_pass ["Student ID"].count()

# Now calculate the percentage of students that passed math/reading/both
pct_math = (count_math / total_students) * 100
pct_read = (count_read / total_students) * 100
pct_both = (count_both / total_students) * 100

#Print the summary results in a DataFrame using a summary dictionary
summary_dicts = [{"Total Schools" : total_schools, 
                 "Total Students" : total_students,
                 "Total Budget" : total_budget,
                 "Average Math Score" : avg_math,
                 "Average Reading Score" : avg_read,
                 "% Passed Math" : pct_math,
                 "% Passed Reading" : pct_read,
                 "% Passed Both" : pct_both}]
summary_df = pd.DataFrame(summary_dicts)

# Format the Data Frame
summary_df["Total Students"] = summary_df["Total Students"].map("{:,}".format)
summary_df["Total Budget"] = summary_df["Total Budget"].map("${:,.2f}".format)
summary_df["Average Math Score"] = summary_df["Average Math Score"].map("{:.5f}".format)
summary_df["Average Reading Score"] = summary_df["Average Reading Score"].map("{:.5f}".format)
summary_df["% Passed Math"] = summary_df["% Passed Math"].map("{:.8}".format)
summary_df["% Passed Reading"] = summary_df["% Passed Reading"].map("{:.8}".format)
summary_df["% Passed Both"] = summary_df["% Passed Both"].map("{:.8}".format)
summary_df

Total Schools Total Students    Total Budget Average Math Score  \
0             15         39,170  $24,649,428.00           78.98537   

  Average Reading Score % Passed Math % Passed Reading % Passed Both  
0              81.87784     74.980853        85.805463     65.172326

In [7]:
# Use Group By to sort by the school name and then display
school_summary_df = school_data_complete_df.groupby(["school_name"])

# Use Group By to get all the column info
school_summary_count = school_summary_df["student_name"].count()
school_summary_type = school_summary_df["type"].unique()
school_summary_budget = school_summary_df["budget"].sum()
school_summary_avg = school_summary_budget / school_summary_count
school_summary_math = school_summary_df["math_score"].mean()
school_summary_read = school_summary_df["reading_score"].mean()

# Use Group By to get data for passing math and passing reading and passing both
pass_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70].groupby("school_name")["Student ID"].count()
pct_pass_math = (pass_math / school_summary_count) * 100
pass_read = school_data_complete_df[school_data_complete_df["reading_score"] >= 70].groupby("school_name")["Student ID"].count()
pct_pass_read = (pass_read / school_summary_count) * 100
pass_both =  school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)].groupby("school_name")["Student ID"].count()
pct_pass_both = (pass_both / school_summary_count) * 100


school_analysis_df = pd.DataFrame({"School Type": school_summary_type,
                                   "Total Students": school_summary_count,
                                   "Total School Budget": school_summary_budget,
                                   "Per Student Budget": school_summary_avg,
                                   "Average Math Score": school_summary_math,
                                   "Average Reading Score": school_summary_read,
                                   "% Passed Math": pct_pass_math,
                                   "% Passed Reading": pct_pass_read,
                                   "% Passed Both": pct_pass_both})
school_analysis_df 

# Format the Data Frame
school_analysis_df["Total School Budget"] = school_analysis_df["Total School Budget"].map("${:,.2f}".format)
school_analysis_df["Per Student Budget"] = school_analysis_df["Per Student Budget"].map("${:,.2f}".format)
school_analysis_df["% Passed Math"] = school_analysis_df["% Passed Math"].map("{:.8}".format)
school_analysis_df["% Passed Reading"] = school_analysis_df["% Passed Reading"].map("{:.8}".format)
school_analysis_df["% Passed Both"] = school_analysis_df["% Passed Both"].map("{:.8}".format)
school_analysis_df

School Type  Total Students Total School Budget  \
school_name                                                             
Bailey High School     [District]            4976  $15,549,641,728.00   
Cabrera High School     [Charter]            1858   $2,009,159,448.00   
Figueroa High School   [District]            2949   $5,557,128,039.00   
Ford High School       [District]            2739   $4,831,365,924.00   
Griffin High School     [Charter]            1468   $1,346,890,000.00   
Hernandez High School  [District]            4635  $14,007,062,700.00   
Holden High School      [Charter]             427     $105,933,149.00   
Huang High School      [District]            2917   $5,573,322,295.00   
Johnson High School    [District]            4761  $14,733,628,650.00   
Pena High School        [Charter]             962     $563,595,396.00   
Rodriguez High School  [District]            3999  $10,186,904,637.00   
Shelton High School     [Charter]            1761   $1,860,672,600.00   
Thomas High School      [Charter]            1635   $1,705,517,550.00   
Wilson High School      [Charter]            2283   $3,012,587,442.00   
Wright High School      [Charter]            1800   $1,888,920,000.00   

                      Per Student Budget  Average Math Score  \
school_name                                                    
Bailey High School         $3,124,928.00           77.048432   
Cabrera High School        $1,081,356.00           83.061895   
Figueroa High School       $1,884,411.00           76.711767   
Ford High School           $1,763,916.00           77.102592   
Griffin High School          $917,500.00           83.351499   
Hernandez High School      $3,022,020.00           77.289752   
Holden High School           $248,087.00           83.803279   
Huang High School          $1,910,635.00           76.629414   
Johnson High School        $3,094,650.00           77.072464   
Pena High School             $585,858.00           83.839917   
Rodriguez High School      $2,547,363.00           76.842711   
Shelton High School        $1,056,600.00           83.359455   
Thomas High School         $1,043,130.00           83.418349   
Wilson High School         $1,319,574.00           83.274201   
Wright High School         $1,049,400.00           83.682222   

                       Average Reading Score % Passed Math % Passed Reading  \
school_name                                                                   
Bailey High School                 81.033963     66.680064         81.93328   
Cabrera High School                83.975780     94.133477        97.039828   
Figueroa High School               81.158020     65.988471        80.739234   
Ford High School                   80.746258     68.309602        79.299014   
Griffin High School                83.816757     93.392371        97.138965   
Hernandez High School              80.934412     66.752967        80.862999   
Holden High School                 83.814988     92.505855        96.252927   
Huang High School                  81.182722     65.683922        81.316421   
Johnson High School                80.966394     66.057551        81.222432   
Pena High School                   84.044699     94.594595        95.945946   
Rodriguez High School              80.744686     66.366592        80.220055   
Shelton High School                83.725724     93.867121        95.854628   
Thomas High School                 83.848930     93.272171        97.308869   
Wilson High School                 83.989488     93.867718        96.539641   
Wright High School                 83.955000     93.333333        96.611111   

                      % Passed Both  
school_name                          
Bailey High School        54.642283  
Cabrera High School       91.334769  
Figueroa High School      53.204476  
Ford High School          54.289887  
Griffin High School       90.599455  
Hernandez High School     53.527508  
Holden High School        89.227166  
Huang

In [8]:
top_overall_df = school_analysis_df.sort_values("% Passed Both", ascending=False)
top_overall_df.head()

School Type  Total Students Total School Budget  \
school_name                                                           
Cabrera High School   [Charter]            1858   $2,009,159,448.00   
Thomas High School    [Charter]            1635   $1,705,517,550.00   
Griffin High School   [Charter]            1468   $1,346,890,000.00   
Wilson High School    [Charter]            2283   $3,012,587,442.00   
Pena High School      [Charter]             962     $563,595,396.00   

                    Per Student Budget  Average Math Score  \
school_name                                                  
Cabrera High School      $1,081,356.00           83.061895   
Thomas High School       $1,043,130.00           83.418349   
Griffin High School        $917,500.00           83.351499   
Wilson High School       $1,319,574.00           83.274201   
Pena High School           $585,858.00           83.839917   

                     Average Reading Score % Passed Math % Passed Reading  \
school_name                                                                 
Cabrera High School              83.975780     94.133477        97.039828   
Thomas High School               83.848930     93.272171        97.308869   
Griffin High School              83.816757     93.392371        97.138965   
Wilson High School               83.989488     93.867718        96.539641   
Pena High School                 84.044699     94.594595        95.945946   

                    % Passed Both  
school_name                        
Cabrera High School     91.334769  
Thomas High School      90.948012  
Griffin High School     90.599455  
Wilson High School      90.582567  
Pena High School        90.540541

In [9]:
bottom_overall_df = school_analysis_df.sort_values("% Passed Both", ascending=True)
bottom_overall_df.head()

School Type  Total Students Total School Budget  \
school_name                                                             
Rodriguez High School  [District]            3999  $10,186,904,637.00   
Figueroa High School   [District]            2949   $5,557,128,039.00   
Huang High School      [District]            2917   $5,573,322,295.00   
Hernandez High School  [District]            4635  $14,007,062,700.00   
Johnson High School    [District]            4761  $14,733,628,650.00   

                      Per Student Budget  Average Math Score  \
school_name                                                    
Rodriguez High School      $2,547,363.00           76.842711   
Figueroa High School       $1,884,411.00           76.711767   
Huang High School          $1,910,635.00           76.629414   
Hernandez High School      $3,022,020.00           77.289752   
Johnson High School        $3,094,650.00           77.072464   

                       Average Reading Score % Passed Math % Passed Reading  \
school_name                                                                   
Rodriguez High School              80.744686     66.366592        80.220055   
Figueroa High School               81.158020     65.988471        80.739234   
Huang High School                  81.182722     65.683922        81.316421   
Hernandez High School              80.934412     66.752967        80.862999   
Johnson High School                80.966394     66.057551        81.222432   

                      % Passed Both  
school_name                          
Rodriguez High School     52.988247  
Figueroa High School      53.204476  
Huang High School         53.513884  
Hernandez High School     53.527508  
Johnson High School       53.539172

In [10]:
# Use the loc function and Group By to find all the graded students and their corresponding math scores
math_avg_ninth = school_data_complete_df.loc[school_data_complete_df["grade"] == "9th"].groupby("school_name")["math_score"].mean()
math_avg_tenth = school_data_complete_df.loc[school_data_complete_df["grade"] == "10th"].groupby("school_name")["math_score"].mean()
math_avg_eleventh = school_data_complete_df.loc[school_data_complete_df["grade"] == "11th"].groupby("school_name")["math_score"].mean()
math_avg_twelfth = school_data_complete_df.loc[school_data_complete_df["grade"] == "12th"].groupby("school_name")["math_score"].mean()

# Create a Pandas Data Frame to hold all of this information
math_avg_grade = pd.DataFrame({"Ninth Grade": math_avg_ninth, 
                               "Tenth Grade": math_avg_tenth,
                               "Eleventh Grade": math_avg_eleventh,
                               "Twelfth Grade": math_avg_twelfth})
math_avg_grade

Ninth Grade  Tenth Grade  Eleventh Grade  Twelfth Grade
school_name                                                                   
Bailey High School       77.083676    76.996772       77.515588      76.492218
Cabrera High School      83.094697    83.154506       82.765560      83.277487
Figueroa High School     76.403037    76.539974       76.884344      77.151369
Ford High School         77.361345    77.672316       76.918058      76.179963
Griffin High School      82.044010    84.229064       83.842105      83.356164
Hernandez High School    77.438495    77.337408       77.136029      77.186567
Holden High School       83.787402    83.429825       85.000000      82.855422
Huang High School        77.027251    75.908735       76.446602      77.225641
Johnson High School      77.187857    76.691117       77.491653      76.863248
Pena High School         83.625455    83.372000       84.328125      84.121547
Rodriguez High School    76.859966    76.612500       76.395626      77.690748
Shelton High School      83.420755    82.917411       83.383495      83.778976
Thomas High School       83.590022    83.087886       83.498795      83.497041
Wilson High School       83.085578    83.724422       83.195326      83.035794
Wright High School       83.264706    84.010288       83.836782      83.644986

In [11]:
# Use the loc function and Group By to find all the graded students and their corresponding math scores
read_avg_ninth = school_data_complete_df.loc[school_data_complete_df["grade"] == "9th"].groupby("school_name")["reading_score"].mean()
read_avg_tenth = school_data_complete_df.loc[school_data_complete_df["grade"] == "10th"].groupby("school_name")["reading_score"].mean()
read_avg_eleventh = school_data_complete_df.loc[school_data_complete_df["grade"] == "11th"].groupby("school_name")["reading_score"].mean()
read_avg_twelfth = school_data_complete_df.loc[school_data_complete_df["grade"] == "12th"].groupby("school_name")["reading_score"].mean()

# Create a Pandas Data Frame to hold all of this information
read_avg_grade = pd.DataFrame({"Ninth Grade": read_avg_ninth, 
                               "Tenth Grade": read_avg_tenth,
                               "Eleventh Grade": read_avg_eleventh,
                               "Twelfth Grade": read_avg_twelfth})
read_avg_grade

Ninth Grade  Tenth Grade  Eleventh Grade  Twelfth Grade
school_name                                                                   
Bailey High School       81.303155    80.907183       80.945643      80.912451
Cabrera High School      83.676136    84.253219       83.788382      84.287958
Figueroa High School     81.198598    81.408912       80.640339      81.384863
Ford High School         80.632653    81.262712       80.403642      80.662338
Griffin High School      83.369193    83.706897       84.288089      84.013699
Hernandez High School    80.866860    80.660147       81.396140      80.857143
Holden High School       83.677165    83.324561       83.815534      84.698795
Huang High School        81.290284    81.512386       81.417476      80.305983
Johnson High School      81.260714    80.773431       80.616027      81.227564
Pena High School         83.807273    83.612000       84.335938      84.591160
Rodriguez High School    80.993127    80.629808       80.864811      80.376426
Shelton High School      84.122642    83.441964       84.373786      82.781671
Thomas High School       83.728850    84.254157       83.585542      83.831361
Wilson High School       83.939778    84.021452       83.764608      84.317673
Wright High School       83.833333    83.812757       84.156322      84.073171

In [45]:
# Create bins to hold the spending ranges, numbers on the right are includsive
bins = [0, 583, 629, 644, 675]
bin_names = ["< $584", "$585 - $629", "$630 - $644", "$645 - $675"]
school_data_complete_df["Spending Ranges (Per Student)"] = pd.cut(school_data_complete_df["budget"] / school_data_complete_df["size"], bins, labels=bin_names)
school_data_complete_df["Spending Ranges (Per Student)"]

# Create the Data Frame to display the information, use Group By on the Spending Ranges
spending_df = school_data_complete_df.groupby("Spending Ranges (Per Student)")
avg_math_spending = spending_df["math_score"].mean()
avg_read_spending = spending_df["reading_score"].mean()
pct_math_spending = (school_data_complete_df[school_data_complete_df["math_score"] >= 70].groupby("Spending Ranges (Per Student)")["Student ID"].count() / spending_df["Student ID"].count()) * 100
pct_read_spending = (school_data_complete_df[school_data_complete_df["reading_score"] >= 70].groupby("Spending Ranges (Per Student)")["Student ID"].count() / spending_df["Student ID"].count()) * 100
pct_both_spending = (school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)].groupby("Spending Ranges (Per Student)")["Student ID"].count() / spending_df["Student ID"].count()) * 100

spending_summary_df = pd.DataFrame({"Average Math Score" : avg_math_spending,
                        "Average Reading Score" : avg_read_spending,
                        "% Passed Math" : pct_math_spending,
                        "% Passed Reading" : pct_read_spending,
                        "% Overall Passed" : pct_both_spending})

spending_summary_df

# Format the Data Frame
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.3}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.3}".format)
spending_summary_df["% Passed Math"] = spending_summary_df["% Passed Math"].map("{:.3}".format)
spending_summary_df["% Passed Reading"] = spending_summary_df["% Passed Reading"].map("{:.3}".format)
spending_summary_df["% Overall Passed"] = spending_summary_df["% Overall Passed"].map("{:.3}".format)
spending_summary_df

Average Math Score Average Reading Score  \
Spending Ranges (Per Student)                                            
< $584                                      83.4                  84.0   
$585 - $629                                 80.0                  82.3   
$630 - $644                                 77.8                  81.3   
$645 - $675                                 77.0                  81.0   

                              % Passed Math % Passed Reading % Overall Passed  
Spending Ranges (Per Student)                                                  
< $584                                 93.7             96.7             90.6  
$585 - $629                            79.1             88.5             70.9  
$630 - $644                            70.6             82.6             58.8  
$645 - $675                            66.2             81.1             53.5

In [47]:
# Create bins to hold the size ranges, numbers on the right are includsive
bins = [0, 999, 1999, 5000]
bin_names = ["Small (<1000)", "Medium (1000 - 2000)", "Large (2000 - 5000)"]
school_data_complete_df["School Size"] = pd.cut(school_data_complete_df["size"],bins, labels=bin_names)
school_data_complete_df["School Size"]

# Create the Data Frame to display the information, use Group By on the Size Ranges
size_df = school_data_complete_df.groupby("School Size")
avg_math_size = size_df["math_score"].mean()
avg_read_size = size_df["reading_score"].mean()
pct_math_size = (school_data_complete_df[school_data_complete_df["math_score"] >= 70].groupby("School Size")["Student ID"].count() / size_df["Student ID"].count()) * 100
pct_read_size = (school_data_complete_df[school_data_complete_df["reading_score"] >= 70].groupby("School Size")["Student ID"].count() / size_df["Student ID"].count()) * 100
pct_both_size = (school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)].groupby("School Size")["Student ID"].count() / size_df["Student ID"].count()) * 100

size_summary_df = pd.DataFrame({"Average Math Score" : avg_math_size,
                        "Average Reading Score" : avg_read_size,
                        "% Passed Math" : pct_math_size,
                        "% Passed Reading" : pct_read_size,
                        "% Overall Passed" : pct_both_size})

size_summary_df

# Format the Data Frame
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.3}".format)
size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.3}".format)
size_summary_df["% Passed Math"] = size_summary_df["% Passed Math"].map("{:.3}".format)
size_summary_df["% Passed Reading"] = size_summary_df["% Passed Reading"].map("{:.3}".format)
size_summary_df["% Overall Passed"] = size_summary_df["% Overall Passed"].map("{:.3}".format)
size_summary_df

Average Math Score Average Reading Score % Passed Math  \
School Size                                                                   
Small (<1000)                      83.8                  84.0          94.0   
Medium (1000 - 2000)               83.4                  83.9          93.6   
Large (2000 - 5000)                77.5                  81.2          68.7   

                     % Passed Reading % Overall Passed  
School Size                                             
Small (<1000)                    96.0             90.1  
Medium (1000 - 2000)             96.8             90.6  
Large (2000 - 5000)              82.1             56.6

In [49]:
# Create the Data Frame to display the information, use Group By on the School Type
type_df = school_data_complete_df.groupby("type")
avg_math_type = type_df["math_score"].mean()
avg_read_type = type_df["reading_score"].mean()
pct_math_type = (school_data_complete_df[school_data_complete_df["math_score"] >= 70].groupby("type")["Student ID"].count() / type_df["Student ID"].count()) * 100
pct_read_type = (school_data_complete_df[school_data_complete_df["reading_score"] >= 70].groupby("type")["Student ID"].count() / type_df["Student ID"].count()) * 100
pct_both_type = (school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)].groupby("type")["Student ID"].count() / type_df["Student ID"].count()) * 100

type_summary_df = pd.DataFrame({"Average Math Score" : avg_math_type,
                        "Average Reading Score" : avg_read_type,
                        "% Passed Math" : pct_math_type,
                        "% Passed Reading" : pct_read_type,
                        "% Overall Passed" : pct_both_type})

type_summary_df

# Format the Data Frame
type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.3}".format)
type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.3}".format)
type_summary_df["% Passed Math"] = type_summary_df["% Passed Math"].map("{:.3}".format)
type_summary_df["% Passed Reading"] = type_summary_df["% Passed Reading"].map("{:.3}".format)
type_summary_df["% Overall Passed"] = type_summary_df["% Overall Passed"].map("{:.3}".format)
type_summary_df

Average Math Score Average Reading Score % Passed Math  \
type                                                              
Charter                83.4                  83.9          93.7   
District               77.0                  81.0          66.5   

         % Passed Reading % Overall Passed  
type                                        
Charter              96.6             90.6  
District             80.9             53.7